In [ ]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import scipy
import importlib
import pickle
import psutil 
import gc

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

load data

In [33]:
# training_group = ['eBlinqMayStudyCtrl', 'eBlinqMayStudyTest', 'eBlinqJuneStudyCtrl',
#        'eBlinqJuneStudyTest', 'Pre-Piv-2-training',
#        'Pre-Piv-2-validation', 'eBlinq19c', 'eBlinq20', 'eBlinq22',
#        'eBlinq23', 'eBlinq25', 'eBlinqRingOverlay&NiMNA', 'iBlinqAcet',
#        'eBlinq14', 'eBlinq15', 'eBlinq16', 'eBlinq17', 'eBlinq18',
#        'eBlinq19a', 'eBlinq19b', 'iBlinq16', 'iBlinqAdhesive',
#        'iBlinqFW231', 'iBlinqOverlay']
training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay','Pre-Piv-2-training']
training_time_window = [0.083,5]
training_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide', 'Tier 2: 2/3 Pass CLN', 'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide']
training_config = ['Control', 'Unspecified']
training_chemistry = '2.6.11'
training_ivarm = ['No', 'Unspecified']

test_group = ['eBlinqMayStudyCtrl', 'eBlinqJuneStudyCtrl']
# test_group = ['eBlinqMayStudyCtrl', 'eBlinqJuneStudyCtrl','Pre-Piv-2-validation']
test_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide', 'Tier 2: 2/3 Pass CLN', 'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide']
test_config = ['Control', 'Unspecified']
test_chemistry = '2.6.11'
test_ivarm = ['No', 'Unspecified']

all_training_channel = ['1'] # ['1','3','4']
iso_thresh = 25
feature_hour_short_window = '8hr'
feature_hour_long_window = '120hr'
feature_col = [
    'time_from_start_featureGeneration',
    'Temperature_alignment',
    'CE_alignment',
    'deg1_103_featureGeneration',
    'mag1_103_featureGeneration',
    'deg3_103_featureGeneration',
    'mag3_103_featureGeneration',
    'deg4_103_featureGeneration',
    'mag4_103_featureGeneration',
    'deg1_1007_featureGeneration',
    'mag1_1007_featureGeneration',
    'deg3_1007_featureGeneration',
    'mag3_1007_featureGeneration',
    'deg4_1007_featureGeneration',
    'mag4_1007_featureGeneration',
    'deg1_10000_featureGeneration',
    'mag1_10000_featureGeneration',
    'deg3_10000_featureGeneration',
    'mag3_10000_featureGeneration',
    'deg4_10000_featureGeneration',
    'mag4_10000_featureGeneration',
    'ewm_average_8hr_deg1_10000_featureGeneration',
    'ewm_average_8hr_deg1_1007_featureGeneration',
    'ewm_average_8hr_deg1_103_featureGeneration',
    'ewm_average_8hr_deg3_10000_featureGeneration',
    'ewm_average_8hr_deg3_1007_featureGeneration',
    'ewm_average_8hr_deg3_103_featureGeneration',
    'ewm_average_8hr_deg4_10000_featureGeneration',
    'ewm_average_8hr_deg4_1007_featureGeneration',
    'ewm_average_8hr_deg4_103_featureGeneration',
    'ewm_average_8hr_mag1_10000_featureGeneration',
    'ewm_average_8hr_mag1_1007_featureGeneration',
    'ewm_average_8hr_mag1_103_featureGeneration',
    'ewm_average_8hr_mag3_10000_featureGeneration',
    'ewm_average_8hr_mag3_1007_featureGeneration',
    'ewm_average_8hr_mag3_103_featureGeneration',
    'ewm_average_8hr_mag4_10000_featureGeneration',
    'ewm_average_8hr_mag4_1007_featureGeneration',
    'ewm_average_8hr_mag4_103_featureGeneration',    
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',    
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
]
gluc_col = [
    'model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose',
]

info_ref = ['Study','Sensor_Id','Chemistry mfg','Sorting Tier mfg','IV Arm','Config']
retrofit_col = ['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']
ref_for_optim = 'cgm_adj' # 'ysi', 'acck'
ground_truth_col = [ref_for_optim]
all_cols = feature_col + gluc_col + info_ref + retrofit_col + ground_truth_col

In [ ]:
df_orig=pd.read_csv(r'S:\for Liang\data for optimization\agg_sample_df_ref_gModel_cgmadj_all2.csv')

In [ ]:
sensor_info_df = pd.read_excel(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\neural network glucose model\all sensor info updated.xlsx')

In [36]:
sensor_info_df.columns

Index(['Study', 'sensor_id', 'sensor_id_corrected', 'Subject ID', 'IV Arm',
       'Self Apply', 'Location', 'Config', 'Sorting Tier mfg', 'Chemistry mfg',
       'Lot', 'Build', 'Wafer Lot #', 'Wafer Type', 'Wafer Batch - #',
       'Wafer Location', 'Cardinal', 'Quadrant', 'Edge Level', 'Outermost'],
      dtype='object')

In [ ]:
df_orig = df_orig.merge(right=sensor_info_df,how='left',left_on='Sensor_Id',right_on='sensor_id',suffixes=(None,'2'))

In [37]:
df_orig['Config'].unique()

array(['Control', nan, 'Unspecified'], dtype=object)

In [38]:
df_orig['IV Arm'].unique()

array(['No', 'Yes', nan, 'Unspecified'], dtype=object)

In [39]:
df_orig['Study'].unique()

array(['eBlinqMayStudyCtrl', 'eBlinqMayStudyTest', 'eBlinqJuneStudyCtrl',
       'eBlinqJuneStudyTest', 'Pre-Piv-2-training',
       'Pre-Piv-2-validation', 'eBlinq19c', 'eBlinq20', 'eBlinq22',
       'eBlinq23', 'eBlinq25', 'eBlinqRingOverlay&NiMNA', 'iBlinqAcet',
       'eBlinq14', 'eBlinq15', 'eBlinq16', 'eBlinq17', 'eBlinq18',
       'eBlinq19a', 'eBlinq19b', 'iBlinq16', 'iBlinqAdhesive',
       'iBlinqFW231', 'iBlinqOverlay'], dtype=object)

In [40]:
df_orig['Chemistry mfg'].unique()

array(['2.6.11', nan, '2.5.11'], dtype=object)

In [41]:
df_orig['Sorting Tier mfg'].unique()

array(['Tier 1: 3/3 Clinical Use', nan, 'Tier 1w-a: 2/3 CLN, 3/3 Wide',
       'Tier 5: Non-Functional', 'Tier 2: 2/3 Pass CLN',
       'Tier 1w-b: 1/3 CLN, 3/3 Wide', 'Tier 1w-c: 0/3 CLN, 3/3 Wide'],
      dtype=object)

In [ ]:
df = df_orig[all_cols].copy()

In [47]:
all_results= {}
all_optim_params = {}
hypo_idx = df[ref_for_optim].le(100)
for ch_str in all_training_channel:
    print(f'Optmizing channel {ch_str}')
    this_result = []
    df[f'ad_s{ch_str}'] = df[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df[ref_for_optim]).abs()
    df[f'ard_s{ch_str}'] = df[f'ad_s{ch_str}'].div(df[ref_for_optim]).mul(100)
    df[f'iso_s{ch_str}'] = df[f'ard_s{ch_str}']
    df.loc[hypo_idx,f'iso_s{ch_str}'] = df.loc[hypo_idx,f'ad_s{ch_str}']
    col_for_train = list(df.columns)
    training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                            df['Chemistry mfg'].eq(training_chemistry) & df['Sorting Tier mfg'].isin(training_tier) &\
                            df['IV Arm'].isin(training_ivarm) & df['Config'].isin(training_config)
    len_training = training_index.sum()
    valid_training_index = training_index & df[f'iso_s{ch_str}'].le(iso_thresh)
    len_valid_training = valid_training_index.sum()
    df_training_valid = df.loc[valid_training_index,col_for_train].dropna().reset_index(drop=True)
    print(f'Training data has length {len_training}')
    print(f'Training data has length {len_valid_training}')
    good_data_ratio = len_valid_training/len_training
    print(f'Ratio of good data is {good_data_ratio}')
    x_train = df_training_valid[feature_col]
    y_train = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid[ref_for_optim]).to_numpy()
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    x_train_norm = min_max_scaler.fit_transform(x_train)
    err_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
    err_model.fit(x_train_norm,y_train)
    predicted_err_train = err_model.predict(x_train_norm)
    df_training_valid[f'model_output_s{ch_str}_correction'] = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(predicted_err_train)
    n_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
    mrd_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
    stdrd_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
    mard_train_today = df_training_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
    n_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
    mrd_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
    stdrd_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
    mard_train_correction = df_training_valid[f'model_output_s{ch_str}_correction'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
    n_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
    mrd_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
    stdrd_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
    mard_train_retro = df_training_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
    print(f'Training data for channel {ch_str} glucose by model today N = {n_train_today}, MRD = {mrd_train_today}, StdRD = {stdrd_train_today}, MARD = {mard_train_today}')
    print(f'Training data for channel {ch_str} glucose by selfcal N = {n_train_correction}, MRD = {mrd_train_correction}, StdRD = {stdrd_train_correction}, MARD = {mard_train_correction}')
    print(f'Training data for channel {ch_str} glucose by retro N = {n_train_retro}, MRD = {mrd_train_retro}, StdRD = {stdrd_train_retro}, MARD = {mard_train_retro}')

    test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                            df['Chemistry mfg'].eq(test_chemistry) & df['Sorting Tier mfg'].isin(test_tier) &\
                            df['IV Arm'].isin(test_ivarm) & df['Config'].isin(test_config)    
    valid_test_index = test_index & df[f'iso_s{ch_str}'].le(iso_thresh)
    df_test_valid = df.loc[valid_test_index,col_for_train].dropna().reset_index(drop=True)
    print(f'Test data has shape {df_test_valid.shape}')
    x_test = df_test_valid[feature_col]
    x_test_norm = min_max_scaler.transform(x_test)
    y_test = df_test_valid[ground_truth_col].to_numpy()
    predicted_err_test = err_model.predict(x_test_norm)
    df_test_valid[f'model_output_s{ch_str}_correction'] = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(predicted_err_test)
    n_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
    mrd_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
    stdrd_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
    mard_test_today = df_test_valid[f'model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
    n_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
    mrd_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
    stdrd_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
    mard_test_correction = df_test_valid[f'model_output_s{ch_str}_correction'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
    n_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
    mrd_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
    stdrd_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
    mard_test_retro = df_test_valid[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
    print(f'Test data for channel {ch_str} glucose by model today N = {n_test_today}, MRD = {mrd_test_today}, StdRD = {stdrd_test_today}, MARD = {mard_test_today}')
    print(f'Test data for channel {ch_str} glucose by selfcal N = {n_test_correction}, MRD = {mrd_test_correction}, StdRD = {stdrd_test_correction}, MARD = {mard_test_correction}')
    print(f'Test data for channel {ch_str} glucose by retro N = {n_test_retro}, MRD = {mrd_test_retro}, StdRD = {stdrd_test_retro}, MARD = {mard_test_retro}')


Optmizing channel 1
Training data has length 6675143
Training data has length 3809504
Ratio of good data is 0.5706999835059713


In [ ]:
df = df_orig[all_cols].copy()
feature_name_short_avg_ch1 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_short_avg_ch1_lb = df[feature_name_short_avg_ch1].quantile(q=percentile_lb)
feature_name_short_avg_ch1_ub = df[feature_name_short_avg_ch1].quantile(q=percentile_ub)
feature_name_short_std_ch1 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_short_std_ch1_lb = df[feature_name_short_std_ch1].quantile(q=percentile_lb)
feature_name_short_std_ch1_ub = df[feature_name_short_std_ch1].quantile(q=percentile_ub)
feature_name_short_avg_ch3 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_short_avg_ch3_lb = df[feature_name_short_avg_ch3].quantile(q=percentile_lb)
feature_name_short_avg_ch3_ub = df[feature_name_short_avg_ch3].quantile(q=percentile_ub)
feature_name_short_std_ch3 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_short_std_ch3_lb = df[feature_name_short_std_ch3].quantile(q=percentile_lb)
feature_name_short_std_ch3_ub = df[feature_name_short_std_ch3].quantile(q=percentile_ub)
feature_name_short_avg_ch4 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_short_avg_ch4_lb = df[feature_name_short_avg_ch4].quantile(q=percentile_lb)
feature_name_short_avg_ch4_ub = df[feature_name_short_avg_ch4].quantile(q=percentile_ub)
feature_name_short_std_ch4 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_short_std_ch4_lb = df[feature_name_short_std_ch4].quantile(q=percentile_lb)
feature_name_short_std_ch4_ub = df[feature_name_short_std_ch4].quantile(q=percentile_ub)

feature_name_long_avg_ch1 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_long_avg_ch1_lb = df[feature_name_long_avg_ch1].quantile(q=percentile_lb)
feature_name_long_avg_ch1_ub = df[feature_name_long_avg_ch1].quantile(q=percentile_ub)
feature_name_long_std_ch1 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_long_std_ch1_lb = df[feature_name_long_std_ch1].quantile(q=percentile_lb)
feature_name_long_std_ch1_ub = df[feature_name_long_std_ch1].quantile(q=percentile_ub)
feature_name_long_avg_ch3 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_long_avg_ch3_lb = df[feature_name_long_avg_ch3].quantile(q=percentile_lb)
feature_name_long_avg_ch3_ub = df[feature_name_long_avg_ch3].quantile(q=percentile_ub)
feature_name_long_std_ch3 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_long_std_ch3_lb = df[feature_name_long_std_ch3].quantile(q=percentile_lb)
feature_name_long_std_ch3_ub = df[feature_name_long_std_ch3].quantile(q=percentile_ub)
feature_name_long_avg_ch4 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_long_avg_ch4_lb = df[feature_name_long_avg_ch4].quantile(q=percentile_lb)
feature_name_long_avg_ch4_ub = df[feature_name_long_avg_ch4].quantile(q=percentile_ub)
feature_name_long_std_ch4 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_long_std_ch4_lb = df[feature_name_long_std_ch4].quantile(q=percentile_lb)
feature_name_long_std_ch4_ub = df[feature_name_long_std_ch4].quantile(q=percentile_ub)

rmse_s1_thresh = df['retro_rmse_s1'].quantile(q=rmse_quantile_thresh)
rmse_s3_thresh = df['retro_rmse_s3'].quantile(q=rmse_quantile_thresh)
rmse_s4_thresh = df['retro_rmse_s4'].quantile(q=rmse_quantile_thresh)

training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                df['Chemistry'].eq(training_chemistry) & df['Sorting Tier'].isin(training_tier)
df_training = df.loc[training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df['Chemistry'].eq(training_chemistry) & df['Sorting Tier'].isin(training_tier) & \
                        df[feature_name_short_avg_ch1].between(feature_name_short_avg_ch1_lb,feature_name_short_avg_ch1_ub) & \
                        df[feature_name_short_std_ch1].between(feature_name_short_std_ch1_lb,feature_name_short_std_ch1_ub) & \
                        df[feature_name_short_avg_ch3].between(feature_name_short_avg_ch3_lb,feature_name_short_avg_ch3_ub) & \
                        df[feature_name_short_std_ch3].between(feature_name_short_std_ch3_lb,feature_name_short_std_ch3_ub) & \
                        df[feature_name_short_avg_ch4].between(feature_name_short_avg_ch4_lb,feature_name_short_avg_ch4_ub) & \
                        df[feature_name_short_std_ch4].between(feature_name_short_std_ch4_lb,feature_name_short_std_ch4_ub) & \
                        df[feature_name_long_avg_ch1].between(feature_name_long_avg_ch1_lb,feature_name_long_avg_ch1_ub) & \
                        df[feature_name_long_std_ch1].between(feature_name_long_std_ch1_lb,feature_name_long_std_ch1_ub) & \
                        df[feature_name_long_avg_ch3].between(feature_name_long_avg_ch3_lb,feature_name_long_avg_ch3_ub) & \
                        df[feature_name_long_std_ch3].between(feature_name_long_std_ch3_lb,feature_name_long_std_ch3_ub) & \
                        df[feature_name_long_avg_ch4].between(feature_name_long_avg_ch4_lb,feature_name_long_avg_ch4_ub) & \
                        df[feature_name_long_std_ch4].between(feature_name_long_std_ch4_lb,feature_name_long_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_training_valid = df.loc[valid_training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Training data has shape {df_training.shape}')
print(f'Training data has shape {df_training_valid.shape}')
good_data_ratio = len(df_training_valid)/len(df_training)
print(f'Ratio of training good data is {good_data_ratio}')
x_train = df_training_valid[feature_col]
y_train = df_training_valid[ground_truth_col].to_numpy()
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
x_train_norm = min_max_scaler.fit_transform(x_train)

test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
            df['Chemistry'].eq(test_chemistry) & df['Sorting Tier'].isin(test_tier)
df_test = df.loc[test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df['Chemistry'].eq(test_chemistry) & df['Sorting Tier'].isin(test_tier) & \
                        df[feature_name_short_avg_ch1].between(feature_name_short_avg_ch1_lb,feature_name_short_avg_ch1_ub) & \
                        df[feature_name_short_std_ch1].between(feature_name_short_std_ch1_lb,feature_name_short_std_ch1_ub) & \
                        df[feature_name_short_avg_ch3].between(feature_name_short_avg_ch3_lb,feature_name_short_avg_ch3_ub) & \
                        df[feature_name_short_std_ch3].between(feature_name_short_std_ch3_lb,feature_name_short_std_ch3_ub) & \
                        df[feature_name_short_avg_ch4].between(feature_name_short_avg_ch4_lb,feature_name_short_avg_ch4_ub) & \
                        df[feature_name_short_std_ch4].between(feature_name_short_std_ch4_lb,feature_name_short_std_ch4_ub) & \
                        df[feature_name_long_avg_ch1].between(feature_name_long_avg_ch1_lb,feature_name_long_avg_ch1_ub) & \
                        df[feature_name_long_std_ch1].between(feature_name_long_std_ch1_lb,feature_name_long_std_ch1_ub) & \
                        df[feature_name_long_avg_ch3].between(feature_name_long_avg_ch3_lb,feature_name_long_avg_ch3_ub) & \
                        df[feature_name_long_std_ch3].between(feature_name_long_std_ch3_lb,feature_name_long_std_ch3_ub) & \
                        df[feature_name_long_avg_ch4].between(feature_name_long_avg_ch4_lb,feature_name_long_avg_ch4_ub) & \
                        df[feature_name_long_std_ch4].between(feature_name_long_std_ch4_lb,feature_name_long_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_test_valid = df.loc[valid_test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Test data has shape {df_test.shape}')
print(f'Test data has shape {df_test_valid.shape}')
good_test_data_ratio = len(df_test_valid)/len(df_test)
print(f'Ratio of good test data is {good_test_data_ratio}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

In [ ]:
self_cal_model_dict = {}

In [ ]:
# For channel 1 slope
i = 0
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
# For channel 1 intercept
i = 1
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_training_valid['model_output_s1_selfcal'] = df_training_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_valid['model_output_s1_selfcal'] = df_test_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_train_ysi['model_output_s1_selfcal'] = df_train_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['model_output_s1_selfcal'] = df_test_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

In [ ]:
[ch1_slope_train.min(),
ch1_slope_train.max(),
ch1_intercept_train.min(),
ch1_intercept_train.max(),
ch1_slope_test.min(),
ch1_slope_test.max(),
ch1_intercept_test.min(),
ch1_intercept_test.max()]

In [ ]:
# For channel 3 slope
i = 2
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
# For channel 3 intercept
i = 3
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_training_valid['model_output_s3_selfcal'] = df_training_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_valid['model_output_s3_selfcal'] = df_test_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_train_ysi['model_output_s3_selfcal'] = df_train_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['model_output_s3_selfcal'] = df_test_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

In [ ]:
[ch3_slope_train.min(),
ch3_slope_train.max(),
ch3_intercept_train.min(),
ch3_intercept_train.max(),
ch3_slope_test.min(),
ch3_slope_test.max(),
ch3_intercept_test.min(),
ch3_intercept_test.max()]

In [ ]:
# For channel 4 slope
i = 4
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
# For channel 4 intercept
i = 5
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_training_valid['model_output_s4_selfcal'] = df_training_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_valid['model_output_s4_selfcal'] = df_test_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_train_ysi['model_output_s4_selfcal'] = df_train_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['model_output_s4_selfcal'] = df_test_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

In [ ]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

In [ ]:
[ch4_slope_train.min(),
ch4_slope_train.max(),
ch4_intercept_train.min(),
ch4_intercept_train.max(),
ch4_slope_test.min(),
ch4_slope_test.max(),
ch4_intercept_test.min(),
ch4_intercept_test.max()]

In [ ]:
df_test_valid[['model_output_s1_selfcal','model_output_s3_selfcal','model_output_s4_selfcal']].std(1).quantile(q=0.95)

In [ ]:
df_test_valid[['model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

In [ ]:
df_test_valid[['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

In [ ]:
min_max_scaler.data_min_

In [ ]:
min_max_scaler.feature_names_in_ = np.array(['time_from_start',
                    'ewm_average_8hr_iir_filt_model_output_temp_s1',
                    'ewm_average_8hr_iir_filt_model_output_temp_s3',
                    'ewm_average_8hr_iir_filt_model_output_temp_s4',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s1',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s3',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s4'
                    ],dtype='object')
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\min_max_scaler.p','wb') as fp:
    pickle.dump(min_max_scaler,fp) 

In [ ]:
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch1_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_0'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch1_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_1'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch3_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_2'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch3_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_3'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch4_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_4'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch4_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_5'],fp)                  

In [ ]:
self_cal_model_temp = MLPRegressor(solver='sgd',alpha=1e-1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528,verbose=True)
param_list = {"hidden_layer_sizes": [(10,),(10,10),(10,5,2),(2,5,10),(4,3,4),(3,2,3),(5,5),(5,)]}
gridCV = GridSearchCV(estimator=self_cal_model_temp, param_grid=param_list,cv=10,verbose=True)
gridCV.fit(x_train_norm,y_train[:,i])

In [ ]:
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/min_max_scaler.p','rb') as fp:
#     min_max_scaler = pickle.load(fp)
# min_max_scaler.feature_names_in_ = np.array(['time_from_start_featureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration'],
#       dtype='object')

In [ ]:
# self_cal_model_dict = {}
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_0'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_1'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_2'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_3'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_4'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_5'] = pickle.load(fp)                

In [ ]:
# for i in range(6): # [0,1]:
#     print(f'Loading parameter {i}')
#     self_cal_model = self_cal_model_dict[f'self_cal_model_{i}']
#     predict_slope_intercept = self_cal_model.predict(x_train_norm)
#     mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,i])**2)
#     mse_train_today = np.mean(y_train[:,i]**2)
#     print(f'The model today has MSE={mse_train_today}')
#     print(f'The self cal model has MSE={mse_train_selfcal}')

#     predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
#     mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
#     mse_test_today = np.mean(y_test[:,i]**2)
#     print(f'The model today on test has MSE={mse_test_today}')
#     print(f'The self cal model on test has MSE={mse_test_selfcal}')
#     self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
pd.DataFrame(gridCV.cv_results_)

In [ ]:
gridCV.best_estimator_

In [ ]:
i=1
self_cal_model = MLPRegressor(hidden_layer_sizes=(1,),solver='sgd',alpha=0.1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope_intercept = self_cal_model.predict(x_train_norm)
mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,0])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
y_train[:,i].mean()

In [ ]:
df_training_valid[feature_col].plot(y='ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',kind='hist',bins=1000)

In [ ]:
min_max_scaler.data_max_

In [ ]:
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10),solver='sgd',alpha=1e-4,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,0])
predict_slope_intercept = self_cal_model.predict(x_train_norm)

In [ ]:
valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[f'iso_s{ch_str}'].le(iso_thresh) & df[f'iso_s3'].le(iso_thresh) & df[f'iso_s4'].le(iso_thresh)].index
df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Test data has shape {df_test_valid.shape}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.fit_transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)